# Data visualization of the Observable Universe

In this article, I will make a gentle introduction to making data visualizations on the biggest of scales. Making sense of scientific data you are unfamiliar with teaches the importance of domain knowledge very quickly. Astronomy and cosmology are no exception. Because of that, I will focus more on the theoretical basics than on explaining the Python minutiae. Still, the code is there, and the complete notebook is on my GitHub[1].

Reference plots are at the very end of the notebook.

## The data

You may have heard that the estimated number of galaxies in the observable universe is around two trillion (or several hundred billion, based on the data from the New Horizons). The first thing that is quite important to mention, is that we do not have data for all those galaxies. Only a tiny fraction of those estimates have a data point in astronomy databases. And even less has the location and distance information. The Euclid mission, launched on the 1st of July 2023, will change a lot in this field[2].

For this article's purposes, we will get the data from Hyperleda, an extragalactic database containing the distance information for over 2.8 million individual galaxies.

We can download the data from the HyperLeda website[3] using a simple SQL query:

- v — velocity — will be used to calculate the redshifts,
- modbest — distance modulus — together with redshifts will be used to calculate the distances,
- objtype — ‘Q’ stands for quasars, distant and very bright galaxies, and ‘G’ for more generic, confirmed galaxies. For simplicity, we want only individual galaxies.

We are getting all the columns because we will use them in the future. Here, we will need only the coordinates and the distance. In HyperLeda, the data that has modbest and velocity will always have the coordinates.

We can now load the data.

### Loading the data

The HyperLeda outputs aren't the friendliest of them all, so to open it, we will use the following code first to check the file:

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
%%time
import csv

with open(
    r'./data/HyperLeda_meandata_1686391913.csv', 
    newline='',
) as f:
    head = [next(f) for x in range(95)]
    reader = csv.reader(head)
    data = list(reader)
    
for i, v in enumerate(data):
    print(i, v)

We can see that the header is in line 89, and the data starts in line 91.

Let's now open the data with pandas:

In [ ]:
data = pd.read_csv(
    './data/HyperLeda_meandata_1686391913.csv', 
    header=89, 
    skiprows=[90], 
    low_memory=False,
)

In [ ]:
data = data.iloc[:-5, :].copy() # Standard 5 rows report at the end
data['pgc'] = data['pgc'].astype(int)

Column objtype has some unexpected spaces. Those need to go.

In [ ]:
data.objtype = data.objtype.str.strip()

In [ ]:
data['objtype'].value_counts()

Generic confirmed galaxies are the vast majority of our data.

In [ ]:
print(data.shape)

## Cosmology model

Let's start with the important question:

### Why bother with cosmology at all?

There is a way to spatially visualize that data more simply than we are going to. If you want a cosmology-independent visualization, you can use velocity, or redshift, as a measure of distance. You can even use it to get Euclidean coordinates for 3D plots. The results in terms of distribution will be very similar, except for the units. You won’t have a unit of distance, instead — the radial velocity or redshift. Some astronomers visualize the data this way.

You will also sometimes find the distance calculated by dividing the velocity by the Hubble constant, something along the lines of:

where:

- v is velocity,
- cosmo.H(0) is Hubble constant,
- 1000000 is a multiplier to get megaparsecs.

With this, again, the relative coordinates look similar, but the units of distance would be all wrong. This formula is an approximation and only accurate for relatively close galaxies where the redshift is small. For more distant galaxies, the expansion of the Universe has not been constant over time, and this formula does not account for that. As a result, you find yourself with the radius of the observable universe over twice bigger than current estimations.

We want to make the results as simple to understand as possible, and the scales right. To do that, we will need cosmology.

### Choosing cosmology parameters

We will focus on the currently most widely accepted cosmology model: Lambda Cold Dark Matter, from the family of models called Friedmann-Lemaitre-Robertson-Walker (FLRW) cosmologies. LambdaCDM is a flat, accelerating universe, with a dark energy density parameter of 0.73. This model represents the closest to what we can call the state of our knowledge today. However, there are still things that need to be settled in this model and one of them is the value of the Hubble constant — the rate of the expansion of the Universe.

### Why Hubble tension is a problem

There are two main ways of measuring the expansion rate:

- Standard candles — type 1a supernovae, which occur in binary systems where one star is a white dwarf. They are assumed to have consistent peak luminosity that can be used to measure distance. It’s relatively free from the cosmological model assumptions. So-called ‘late universe’ measurement of the Hubble constant.
- CMB — measurements of the rate of the expansion in the Cosmic Microwave Background. Heavily reliant on LambdaCDM model. So-called “early universe” techniques, they do account for the change in the expansion rate since the early universe.

The most accurate of each of those two types of measurements of the Hubble constant is 73 km/Mpc and 67.7 km/Mpc respectively. While the difference of 5.3 kilometers per 3.26 million lightyears (one megaparsec) doesn’t seem that much, it affects the distance measurement. If we compare:

candles-vs-cmb.png (with images, notebook is to big to display on GitHub. Check reference plots at the end of the notebook.)

Hubble tension (by the author)

On redshift 2, there is already over 1 billion lightyears difference in comoving distance and over 700 million years of light travel time.

The problem is, we don’t know which of those is correct (if any). This discrepancy is called Hubble tension[4] and has been hailed as a crisis in cosmology[5].

For the sake of this exercise, we will go with the value in the middle, used in some calculations in HyperLeda.

### Python libraries

To set up the cosmology we will use Astropy. This library will also help us with coordinates, units, and lots of calculations. It has specific table functions based on NumPy, in a way similar to Pandas. The built-in functions are slightly different from what I am used to, but thanks to methods ‘from_pandas’ and ‘to_pandas’, if you are careful, you can move between them seamlessly.

In [ ]:
import astropy

import astropy.coordinates as coord
import astropy.units as u
from astropy.io import ascii
from astropy.coordinates import SkyCoord

from astropy import constants as const # For speed of light

from astropy.cosmology import FlatLambdaCDM

Instead of importing built-in cosmology[6], we will set the cosmology parameters directly. We will base them on the parameters used to derive distance modulus[7] in the HyperLeda database, and name it ‘HyperLeda’:

In [ ]:
cosmo = FlatLambdaCDM(name='HyperLeda', H0=70, Om0=0.27)
cosmo

The cosmology model we chose is LambdaCDM, with a Hubble constant of 70 km per megaparsec (3.26 million lightyears).

## Coordinates

### Choosing the perspective of a sphere

When visualizing astronomical objects, we can do it in two different ways:

- The view from the center of the sphere — we look up from the rotating earth and see the celestial sphere. Then we try to put this sphere on a 2D surface, which is impossible without distorting the shapes. To minimize the distortion, we can use one of the popular projections, like Aitoff[8], Hammer[9], and Mollweide[10]. Those are from the 19th century, used to represent the surface of the Earth, and adopted to represent the map of the sky. Each of them distorts the image a bit differently.
- The view from the ‘outside’, in three dimensions — 3D projection. If we try to visualize not just a slice, but everything we have in the data, the Earth, the center of our observable universe, will be in the middle of the visualization.

Hyperleda contains three types of astronomical coordinates:

- al2000, de2000 — right ascension and declination (coordinates on the celestial sphere, equivalent of longitude and latitude),
- l2, b2 — galactic longitude and latitude (coordinates in galactic frame),
- sgl, sgb — super-galactic longitude and latitude (coordinates in super-galactic frame).

### Galactic vs supergalactic frame

The difference between the galactic and supergalactic frame is the angle and location of the Milky Way galaxy, which obscures other galaxies and is visible as a dark line.

In the galactic coordinate system, the Milky Way is positioned horizontally in the middle (galactic plane):

galactic.png (with images, notebook is to big to display on GitHub. Check reference plots at the end of the notebook.)

Galactic frame, with Milky Way as a horizontal dark line in the middle along the ‘equator’ (by the author)

The supergalactic coordinate system became a thing in the 1950s with the discovery of the supergalactic plane[11]. The supergalactic coordinate system has the Milky Way (called in extra-galactic astronomy “zone of avoidance”[12], something that obscures the view) vertically in the middle. Because of that, it takes much less space in the projection and most galaxies are in less-distorted regions. This allows us to see extra-galactic (super-galactic) structures in greater detail, than in a galactic coordinate system:

supergalactic.png (with images, notebook is to big to display on GitHub. Check reference plots at the end of the notebook.)

Supergalactic frame, with Milky Way obscuring the middle nearly vertical line between the ‘poles’ (by the author)

Here we will use Aitoff, an azimuthal map projection, to represent the sky in supergalactic coordinates.

While we could make a simplified 2D visualization straight away, we will require 3D coordinates to make a 3D image. And for that we need distance.

## Distance

In astronomy, you will rarely encounter the term without a preceding adjective, and when you do, the distance type must be clear from the context. Luminosity distance, comoving distance, and look-back distance (just to name a few) are very different things and used for different purposes. Just to demonstrate the difference between the three for a redshift of 7:

Luminosity distance in the Milky Way is a solid approximation of the distance in Euclidean space. However, in extra-galactic distances, it’s affected by redshift, time dilation, and space curvature and doesn’t mean much. This luminosity distance for Redshift 7 would be over 5 times higher than the estimated radius of the observable universe. We are going to largely ignore luminosity distance.

Lookback distance (or light travel distance) is the light travel time, which means the light from a galaxy at redshift 7, that we see now, left this galaxy 13.07 billion years ago.

Comoving distance is equal to the proper distance at the current age of the Universe but will differ in the past and the future. It is called comoving because objects move along with the expansion of the universe. Simply put, it is a distance between the objects right now. If in light years, it is a distance the light would travel from one object to the other if the light is emitted now and the universe doesn’t expand till it reaches the destination.

On the biggest of scales, light travel distance is usually much shorter than the comoving distance, because the universe is expanding, and when the light was emitted, the object was much closer to us than it is now[13][14].

As a measure of distance in the HyperLeda data, we could use:

- v— heliocentric radial velocity, defined as ‘cz’ — redshift multiplied by the speed of light[15],
- mod0— distance modulus from distance measurements (redshift-independent)[16],
- modbest — distance modulus related to luminosity distance, a weighted average between redshift-independent and redshift-based distance measurements[17].

Ideally, we would like to use a redshift-independent distance modulus. However, measuring the distance independently from redshift is not easy and quite time-consuming, which is reflected by the number of observations in our data:

In [ ]:
data['mod0'].notna().sum()

That’s not too much to work with, so we will leave it alone on the observable universe scale.

We are going to use ‘modbest’ as the base of our calculations. But we are also going to need ‘v’ to calculate redshift.

### Calculating distance

Now that we have the data initially prepared, we need to do some calculations.

First, let’s convert Pandas DataFrame to Astropy QTable:

In [ ]:
data = astropy.table.QTable.from_pandas(data)

#### Redshift from v

As we can see in the Hyperleda docs, velocity is a redshift, multiplied by the speed of light. So let’s get the redshift itself.

The speed of light is a constant in astropy (imported as const).

In [ ]:
print(const.c)

As constant c is in m/s, it has to be converted to km/s. With this, we can calculate:

In [ ]:
data['redshift'] = data['v'] / const.c.to('km/s').value

Normally redshift column would simply be called ‘z’, but later we are going to have coordinates ‘x’, ‘y’, and z’, so instead of renaming 3 columns, let’s rename this one.

The highest redshift in our data is[18]:

In [ ]:
data['redshift'].max()

Now that we have redshift, we could just calculate the comoving distance straight from the redshift:

In [ ]:
data['dm'] = cosmo.comoving_distance(data['redshift'])

his would be completely okay in the largest of scales, but it would give us a negative distance for the negative redshift (blueshift) — objects gravitationally bound with the Milky Way. So if you would like to see our local neighborhood, it would be severely distorted.

So let’s do it right and use modbest.

#### Distance from distance modulus

First, we will need to convert the distance modulus, set in magnitude (luminosity) to actual distance (distance modulus is the apparent magnitude — the absolute magnitude — for an object at a given redshift)[19].

In [ ]:
data['dl'] = coord.Distance(
    distmod=data['modbest'], 
    unit='Mpc'
)
# distance_modulus = 5 log(DL) + 25

Now that we have our luminosity distance (DL), let´s calculate comoving distance, using the formula:

comoving distance DM = DL/(1+z))

In [ ]:
data['dm'] = data['dl'] / (1 + data['redshift'])

Now that we have a comoving distance, we must decide on the unit — this can be light years, parsecs, or even kilometers. The last one would be a number so big, that it would be almost meaningless. I think the easiest way to represent cosmic distances is in light years. Most people will be familiar with it, and unlike parsec, the light year has some easily identifiable meaning — a distance that light travels in a year. It’s a well-known concept. This familiarity gets a bit shaky when it’s used in an accelerating expanding universe, but still.

Light years will be our unit of distance. But as we talk about the entire observable universe, we will add a “giga” prefix to it, meaning billion. That way, we can talk about the size of the Observable Universe in billions of lightyears and represent it neatly with two-digit numbers.

In [ ]:
# Convert Megaparsecs to Giga Light-years
data['dm'] = data['dm'].to(u.Glyr)

Let's check how far is our farthest object:

In [ ]:
max(data['dm'])

redshift-dm.png (with images, notebook is to big to display on GitHub. Check reference plots at the end of the notebook.)

Relationship between comoving distance and the redshift (by the author)

Which means that the distance from Milky Way to the galaxies at redshift 1 is greater than from galaxies at redshift 1 to redshift 2, and so on.

## Getting 3d coordinates

To visualize our data in 3D, we will need to convert right ascension, declination, and distance from spherical coordinates to coordinates in Euclidean space.

We will apply a simple formula[20] to do that:

x = distance * cos(declination) * cos(right ascension)

y = distance * cos(declination) * sin(right ascension)

z = distance * sin(declination)

Let’s apply it:

In [ ]:
data['x'] = data['dm'] * \
            np.cos(np.radians(data['de2000'])) * \
            np.cos(np.radians(data['al2000'] * 15))
data['y'] = data['dm'] * \
            np.cos(np.radians(data['de2000'])) * \
            np.sin(np.radians(data['al2000'] * 15))
data['z'] = data['dm'] * np.sin(np.radians(data['de2000']))

The coordinates in this process inherit the data type from comoving distance, becoming ‘distance’ themselves. Since ‘distance’ can’t be negative, we will have to convert them to normal values.

In [ ]:
data['x'] = data['x'].value
data['y'] = data['y'].value
data['z'] = data['z'].value

## Adding the Milky Way galaxy

HyperLeda is an extragalactic database, so we shouldn’t expect our home galaxy to be in there. To visualize our local group in the future, we may want to add the Milky Way to our data:

In [ ]:
mw = {'pgc': [0], 
      'objname': ['milkyway'], 
      'objtype': ['G'], 
      't': [3], 
      'type': ['SBb'], 
      'bar': ['B'], 
      'ring': [np.NaN], 
      'al2000': [0],
      'de2000': [0], 
      'v': [0], 
      'modbest': [0], 
      'mod0': [0], 
      'l2': [0], 
      'b2': [0], 
      'sgl': [0], 
      'sgb': [0],
      'redshift': [0], 
      'dm': [0] * u.Glyr, 
      'x': [0],
      'y': [0],
      'z': [0],
     }

mw = astropy.table.QTable(mw)

mw['ring'] = mw['ring'].astype(str)

# Appending Milky Way dataframe to our data
data = astropy.table.vstack([data, mw])

With this, our data is ready. For now.

Let's save it. To save ourselves some space (and time), we will save it as a parquet file. For that, you will need either a pyarrow library or a fastparquet.

In [ ]:
data.write(
    './data/data-vis-obj-univ.parquet', 
    format='parquet', 
    overwrite=True,
)

Let’s take a quick look at the distance distribution in our data:

In [ ]:
# First, we need to load the matplotlib library...
import matplotlib.pyplot as plt

# ...and set the style we are going to use. 
plt.style.use('dark_background')

plt.hist(data['dm'].value, bins=28)
plt.xlabel('Distance')
plt.ylabel('Frequency')
plt.title('Distribution of Distances')
plt.savefig('./plots/distance-distribution.png', dpi=300)
plt.show()

Distribution of distances (by the author)

## Visualizations 2D

Now, let’s do some visual exploring.

We can start with all of our data points in supergalactic coordinates.

In [ ]:
gal = SkyCoord(
    data['sgl'], 
    data['sgb'], 
    frame='supergalactic', 
    unit=u.deg,
)

fig = plt.figure(figsize=(20, 15))
ax = fig.add_subplot(
    111, 
    projection='aitoff', 
    label='galaxies',
)

ax.grid(alpha=0.5)

img = ax.scatter(
    gal.sgl.wrap_at('180d').radian, # it has to be in radians
    gal.sgb.radian, 
    s=0.01,
    c=data['dm'], 
    cmap=plt.cm.viridis_r, # reversed, as short distance dominate
    alpha=None,
    linestyle='None',
);

fig.colorbar(
    img, 
    orientation ='horizontal', 
    aspect=55, 
    label='dm',
    pad=0.07, 
    fraction=0.020,
);

plt.savefig('./plots/all-galaxies-2d.png', dpi=300)

All the data (2.8 million galaxies) in the Aitoff Projection (by the author)

Edges and the middle are dark, obscured by the Milky Way. Some parts are so dense, that in 2D representation we would need a much, much bigger picture to peer through it.

Or, a lot of smaller pictures. Or an animation:

In [ ]:
%%time

import matplotlib.animation as animation
from IPython.display import HTML

# Create a new figure
fig, ax = plt.subplots(
    figsize=(20, 15), 
    subplot_kw={'projection': 'aitoff'},
)

def update(i):
    # Clear the current plot
    ax.cla()
    ax.grid(alpha=0.5)

    # Determine the dm range for this frame
    if i < 19:
        min_dm, max_dm = i * 0.5, (i + 1) * 0.5
    else:
        min_dm, max_dm = 9.5, 28

    # Filter data
    data_dm = data[
        (data['dm'].value > min_dm) 
        & (data['dm'].value <= max_dm)
    ]
    
    # Create scatter plot
    gal = SkyCoord(
        data_dm['sgl'], 
        data_dm['sgb'], 
        frame='supergalactic', 
        unit=u.deg,
    )
    img = ax.scatter(
        gal.sgl.wrap_at('180d').radian, 
        gal.sgb.radian, 
        s=0.01,
        c=data_dm['dm'], 
        cmap=plt.cm.viridis_r,
        alpha=None,
        linestyle='None',
    )
    
    # Set the title
    ax.set_title(
        f'Distance Range: {min_dm} - {max_dm} billion lightyears'
    )

    return img,

# Create the animation
ani = animation.FuncAnimation(
    fig, 
    update,
    frames=range(20), 
    interval=2000, 
    blit=True,
)

# Save the animation
ani.save('./plots/animation-2d.gif', writer='pillow')

# Close the figure
plt.close(fig)

animation-2d.gif (with animations, notebook is to big to display on GitHub.)

Animation showing galaxies in supergalactic frame on Aitoff Projection at different distance ranges (by the author)

## Visualizations 3D

Let's see this in 3d.

In [ ]:
# Setting some 3D options for clarity of the image

def set_3d_plot_options(ax):
    ax.xaxis.pane.fill = False
    ax.yaxis.pane.fill = False
    ax.zaxis.pane.fill = False
    
    ax.xaxis.pane.set_edgecolor('w')
    ax.yaxis.pane.set_edgecolor('w')
    ax.zaxis.pane.set_edgecolor('w')

    ax.grid(False)

In [ ]:
fig = plt.figure(figsize = (20,20))
ax = plt.axes(projection='3d')

set_3d_plot_options(ax)
    
ax.scatter3D(
    data['x'], 
    data['y'], 
    data['z'], 
    s=0.1, 
    c='white', 
    alpha=None,
);

plt.savefig('./plots/all-galaxies-3d.png', dpi=300)

All the data (2.8 million galaxies) in 3D Projection (by the author)

This is the expanse of our data. 2,868,898 confirmed galaxies in the HyperLeda database.

You can see the specific dent in this otherwise quite homogeneous structure. This is the before-mentioned "zone of avoidance" - where the Milky Way, our galaxy, obscures our view. Whatever we know about the galaxies from this region of the sky, doesn't come in visible spectrum.

Noticeable in this image are long lines of galaxies, going back straight to the middle of the plot (Earth). They are called "fingers of God", and are a known issue when using only redshift as a measure of distance. It is a distortion of view caused by gravity affecting the redshifts and apparent density of galaxies in clusters[21]. With only 4039 redshift-independent measurements, we will have to live with the "fingers" here.

# A closer look

Using this data, we can take a closer look and witness the structure emerging from homogeneity, as we move from the edge toward the center of the observable universe:

In [ ]:
%%time

# A fair warning - running this takes about 37 mins

max_dms = [x for x in np.arange(28, 0, -1)] + \
          [round(x, 1) for x in np.arange(0.9, 0, -0.1)]

# Create a new figure
fig = plt.figure(figsize = (20,20))
ax = plt.axes(projection='3d')

def update(max_dm):
    # Clear the current plot
    ax.cla()
    ax.grid(alpha=0.5)

    # Determine the dm range for this frame

    # Filter data
    data_dm = data[(data['dm'].value <= max_dm)]
    
    set_3d_plot_options(ax)

    ax.scatter3D(
        data_dm['x'], 
        data_dm['y'], 
        data_dm['z'], 
        s=0.1, 
        c='white', 
        alpha=None,
    );

    # Set the title
    ax.set_title(
        f'Observable Universe up to {max_dm} billion lightyears'
    )

    return img,

# Create the animation
ani = animation.FuncAnimation(
    fig, 
    update, 
    frames=max_dms, 
    interval=2000, 
    blit=True,
)

# Save the animation
ani.save('./plots/animation-3d.gif', writer='pillow')

# Close the figure
plt.close(fig)

animation-3d.gif (with animations, notebook is to big to display on GitHub.)

Animation of the observable universe in 3D at different distances (by the author)

At around 300 million lightyears we can observe the end of homogeneity of the galaxy distribution. It is an observational scale called 'The End of Greatness'[22].

# Conclusion

In this exercise, we've only scratched the surface of visualizing the observable universe. There is so much more that we can do with that data. We can use lookback distance and treat cosmology as a historical science. We can visualize galaxy types, show the galactic web and galaxy clusters in greater detail, visualize the local group… And it would also be just a start. But isn't it always the case, when talking about space?

Thank you for reading.

# References & further reading

Links to Wikipedia references are mostly self-explanatory.

[1] The repo with the full notebook on GitHub: https://github.com/jan-niedospial/data-vis-obs-univ.

[2] You can read more about the Euclid Mission here: https://www.esa.int/Science_Exploration/Space_Science/Euclid.

[3] https://leda.univ-lyon1.fr/fullsql.html. At the time of this writing, I'm yet to find a reliable way to get all the data that we need from HyperLeda through API.

[4] https://en.wikipedia.org/wiki/Hubble's_law.

[5] You can read about that on Big Think: https://bigthink.com/hard-science/hubble-tension-cosmology-crisis/.

Some of the great articles explaining Hubble tension consequences in simple terms: https://phys.org/news/2023-09-early-dark-energy-hubble-tension.amp by Brian Koberlein. There is also a great article on the topic by Ethan Siegel from 'Starts with the Bang' in Big Think, which explains the cosmic distance ladder very nicely: https://bigthink.com/starts-with-a-bang/hubble-tension-real-solution/. This interview: https://cerncourier.com/a/exploring-the-hubble-tension/ with Licia Verde talks more about the model dependency of different measurement methods.

[6] You can read more about setting up cosmology parameters: https://docs.astropy.org/en/stable/cosmology/index.html.

[7] More on distance modulus on HyperLeda docs: https://leda.univ-lyon1.fr/leda/param/modz.html.

[8] https://en.wikipedia.org/wiki/Aitoff_projection.

[9] https://en.wikipedia.org/wiki/Hammer_projection.

[10] https://en.wikipedia.org/wiki/Mollweide_projection.

[11] https://en.wikipedia.org/wiki/Supergalactic_coordinate_system.

[12] https://en.wikipedia.org/wiki/Zone_of_Avoidance.

[13] If you want to learn more, check this article on Wikipedia: https://en.wikipedia.org/wiki/Comoving_and_proper_distances.

[14] If you want to learn more about cosmology, distances, and redshift by doing, there is a great cosmological calculator:  https://www.astro.ucla.edu/~wright/CosmoCalc.html, written by Ned Wright. You can experiment by inputting different Big Bang cosmological model parameters and choosing different types of the universe model, and you can see how much this can influence distances for any given redshift and see the relationships between different variables. This will help you understand how this all comes together from a theoretical perspective. There is also a Python version of this calculator, which can be easily converted into a function, so if you prefer something less out of the box than 'Astropy', you can use that. The calculator has a very nice glossary too: https://www.astro.ucla.edu/~wright/glossary.html#CC. The scientific paper accompanying it:
Wright, E. L. (2006). A Cosmology Calculator for the World Wide Web. Publications of the Astronomical Society of the Pacific, 118(849), 1711. https://doi.org/10.1086/510102
Finally, if you want to learn about the physics and math behind all of this, you can start by reading about Friedmann's equations: https://en.wikipedia.org/wiki/Friedmann_equations.

[15] Velocity in HyperLeda docs: https://leda.univ-lyon1.fr/leda/param/v.html.

[16] Redshift-independent distance modulus in HyperLeda docs: https://leda.univ-lyon1.fr/leda/param/mod0.html.

[17] Modbest in HyperLeda docs: https://leda.univ-lyon1.fr/leda/param/modbest.html.

[18] If you follow discoveries from the James Webb Space Telescope, you may have heard about galaxy JADES-GS-z13–0 with a redshift of z=13.1. Before James Webb, the highest confirmed redshift was GN-z11 with z=11.09 (after review from JWST it is z=10.6034). In HyperLeda, we don't have such distant objects (at least not with the necessary coordinates and velocity data). You can still add them if you like, e.g. the way we added the Milky Way.

[19] If you want to know more about luminosity distance, or how to manually calculate distance from distance modulus, you can find it on the HyperLeda website: https://leda.univ-lyon1.fr/leda/param/modz.html.

[20] You can find more about the coordinates equations here: https://en.wikipedia.org/wiki/Spherical_coordinate_system.

[21] More on the Fingers of God effect here: https://en.wikipedia.org/wiki/Redshift-space_distortions.

[22] More on the End of Greatness: https://en.wikipedia.org/wiki/Observable_universe#End_of_Greatness.

# Reference plots

### Distance types comparison

In [ ]:
from astropy.cosmology import FlatLambdaCDM

cosmo = FlatLambdaCDM(name='HyperLeda', H0=70, Om0=0.27)

from astropy.cosmology import WMAP9 as wmap9

print(cosmo)
print(wmap9)

In [ ]:
cosmo.Ode0

In [ ]:
import astropy.units as u

redshift = 7

print(cosmo.lookback_distance(redshift).to(u.Glyr))
print(cosmo.comoving_distance(redshift).to(u.Glyr))
print(cosmo.luminosity_distance(redshift).to(u.Glyr))


### Surface tension

In [ ]:
candles = FlatLambdaCDM(name='HyperLeda', H0=73, Om0=0.3)
cmb = FlatLambdaCDM(name='HyperLeda', H0=67.7, Om0=0.3)

In [ ]:
import pandas as pd

cosmologies = pd.DataFrame({
    'candles': [
        candles.lookback_distance(redshift).to(u.Glyr), 
        candles.comoving_distance(redshift).to(u.Glyr), 
        candles.luminosity_distance(redshift).to(u.Glyr),
    ],
    'cmb': [
    cmb.lookback_distance(redshift).to(u.Glyr),
    cmb.comoving_distance(redshift).to(u.Glyr),
    cmb.luminosity_distance(redshift).to(u.Glyr),
    ],
}, index=[
    'lookback_distance', 
    'comoving_distance', 
    'luminosity_distance']
)

In [ ]:
cosmologies

In [ ]:
import astropy

import astropy.coordinates as coord
import astropy.units as u
from astropy.io import ascii
from astropy.coordinates import SkyCoord

from astropy.cosmology import FlatLambdaCDM

from astropy import constants as const # For speed of light

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

plt.style.use('dark_background')

In [ ]:
data = astropy.table.QTable.read(
    './data/data-vis-obj-univ.parquet',
)

In [ ]:
candles = FlatLambdaCDM(name='HyperLeda', H0=73, Om0=0.3)
cmb = FlatLambdaCDM(name='HyperLeda', H0=67.7, Om0=0.3)

data['dm_candles'] = candles.comoving_distance(data['redshift']).to(u.Glyr)
data['dm_cmb'] = cmb.comoving_distance(data['redshift']).to(u.Glyr)

data['tl_candles'] = candles.lookback_distance(data['redshift']).to(u.Glyr)
data['tl_cmb'] = cmb.lookback_distance(data['redshift']).to(u.Glyr)



In [ ]:
df = data.to_pandas()

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)

# Create a figure and two subplots
fig, ax = plt.subplots(1, 2, figsize=(15, 7.5), )

bins = np.linspace(
    df['redshift'].min(), 
    df['redshift'].max(), 
    num=100,
)

# Bin df and compute means and standard deviations
df['bin'] = pd.cut(df['redshift'], bins=bins)
bin_means = df.groupby('bin').mean(numeric_only=True)
bin_stds = df.groupby('bin').std(numeric_only=True)

# Plot means with error bars in the first subplot
ax[0].errorbar(
    x=bin_means['redshift'], 
    y=bin_means['dm_candles'], 
    label='Candles',
)
ax[0].errorbar(
    x=bin_means['redshift'], 
    y=bin_means['dm_cmb'], 
    label='CMB',
)
ax[0].set_xlabel('Redshift', fontsize=12)
ax[0].set_ylabel(
    'Comoving Distance (in giga-lightyears)', 
    fontsize=12,
)
ax[0].yaxis.set_minor_locator(AutoMinorLocator())
ax[0].tick_params(which='major', length=7)
ax[0].tick_params(which='minor', length=4)
for i in ax[0].get_yticklabels():
    i.set_fontsize(12)
for i in ax[0].get_xticklabels():
    i.set_fontsize(12)
ax[0].grid(alpha=0.3, which='both')
ax[0].legend(fontsize=12)


# Bin df and compute means and standard deviations
df['bin'] = pd.cut(df['redshift'], bins=bins)
bin_means = df.groupby('bin').mean(numeric_only=True)
bin_stds = df.groupby('bin').std(numeric_only=True)

# Plot means with error bars in the second subplot
ax[1].errorbar(
    x=bin_means['redshift'], 
    y=bin_means['tl_candles'], 
    label='Candles',
)
ax[1].errorbar(
    x=bin_means['redshift'], 
    y=bin_means['tl_cmb'], 
    label='CMB',
)
ax[1].set_xlabel('Redshift', fontsize=12)
ax[1].set_ylabel(
    'Look Back Distance (in giga-lightyears)', 
    fontsize=12,
)
ax[1].yaxis.set_minor_locator(AutoMinorLocator(n=2))
ax[1].tick_params(which='major', length=7)
ax[1].tick_params(which='minor', length=4)
for i in ax[1].get_yticklabels():
    i.set_fontsize(12)
for i in ax[1].get_xticklabels():
    i.set_fontsize(12)
ax[1].grid(alpha=0.3, which='both')
ax[1].legend(fontsize=12)

plt.tight_layout()

plt.savefig('./plots/reference-plots/candles-vs-cmb.png')

plt.show()

### Redshift vs comoving distance

In [ ]:
# First, we need to load the matplotlib library...

import matplotlib.pyplot as plt

# ...and set the style we are going to use. 

plt.style.use('dark_background')

df = data[['redshift', 'dm']].to_pandas()

bins = np.linspace(
    df['redshift'].min(), 
    df['redshift'].max(), 
    num=100
)

# Bin df and compute means and standard deviations
df['bin'] = pd.cut(df['redshift'], bins=bins)
bin_means = df.groupby('bin').mean()
bin_stds = df.groupby('bin').std()

fig = plt.figure(figsize=(10, 8))

# Plot means with error bars
plt.errorbar(
    x=bin_means['redshift'], 
    y=bin_means['dm'], 
    label='Redshift vs Comoving Distance',
)
#plt.axhline(y = 46.5, color = 'y', linestyle = '-', label='Particle Horizon')
plt.xlabel('Redshift')
plt.ylabel('Comoving Distance')
plt.legend()

plt.savefig('./plots/reference-plots/redshift-dm.png', dpi=300)

plt.show()

In [ ]:
data_05 = data[(data['dm'].value <= 0.5)]

In [ ]:
gal = SkyCoord(
    data_05['l2'], 
    data_05['b2'], 
    frame='galactic', 
    unit=u.deg,
)

fig = plt.figure(figsize=(20, 15))

ax = fig.add_subplot(
    111, 
    projection='aitoff', 
    label='galaxies',
)

ax.grid(alpha=0.5)

img = ax.scatter(
    gal.l.wrap_at('180d').radian, 
    gal.b.radian, 
    s=0.01,
    c=data_05['dm'], 
    cmap=plt.cm.viridis,
    alpha=None,
    linestyle='None',
);

plt.savefig('./plots/reference-plots/galactic.png', dpi=300)

In [ ]:
gal = SkyCoord(data_05['sgl'], data_05['sgb'], frame='supergalactic', unit=u.deg)

fig = plt.figure(figsize=(20, 15))

ax = fig.add_subplot(
    111, 
    projection='aitoff', 
    label='galaxies',
)

ax.grid(alpha=0.5)

img = ax.scatter(
    gal.sgl.wrap_at('180d').radian, 
    gal.sgb.radian, 
    s=0.01,
    c=data_05['dm'], 
    cmap=plt.cm.viridis,
    alpha=None,
    linestyle='None',
);


plt.savefig('./plots/reference-plots/supergalactic.png', dpi=300)